Using Pyspark to read the dataset and check missing values.

In [1]:
# import packages
from pyspark.sql import SparkSession
from pyspark.ml import feature
from pyspark.ml import classification
from pyspark.sql import functions as fn
from pyspark.sql.functions import isnan, when, count, col
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, \
    MulticlassClassificationEvaluator, \
    RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pyspark.sql import SparkSession


spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
# expand the output display to see more columns of a pandas DataFrame
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
# check the shape of df - use this idiots
def size_shape(df):
    
    print("rows:",df.count())
    print("columns:",len(df.columns))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [3]:
# read the dataset
df = spark.read.csv('C:\\Users\\tigerman381\\Downloads\\lending-club-loan-data\\loan.csv', header=True, inferSchema=True)

In [4]:
# take a look at the first 10 rows
df_pd = df.limit(10).toPandas()
display(df_pd.head())

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,None,None,2500,2500,2500.0,36 months,13.56,84.92,C,C1,Chef,10+ years,RENT,55000,Not Verified,Dec-2018,Current,n,None,None,debt_consolidation,Debt consolidation,109xx,NY,18.24,0,Apr-2001,1,None,45,9,1,4341,10.3,34,w,2386.02,2386.02,167.02,167.02,113.98,53.04,0.0,0.0,0.0,Feb-2019,84.92,Mar-2019,Feb-2019,0,None,1,Individual,None,None,None,0,0,16901,2,2,1,2,2,12560,69,2,7,2137,28,42000,1,11,2,9,1878,34360.0,5.9,0.0,0,140.0,212,1,1,0,1.0,NaN,2,NaN,0,2,5,3,3,16,7,18,5,9,0,0,0,3,100.0,0.0,1,0,60124,16901,36500,18124,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Cash,N,None,None,None,None,None,None
1,None,None,30000,30000,30000.0,60 months,18.94,777.23,D,D2,Postmaster,10+ years,MORTGAGE,90000,Source Verified,Dec-2018,Current,n,None,None,debt_consolidation,Debt consolidation,713xx,LA,26.52,0,Jun-1987,0,71,75,13,1,12315,24.2,44,w,29387.75,29387.75,1507.11,1507.11,612.25,894.86,0.0,0.0,0.0,Feb-2019,777.23,Mar-2019,Feb-2019,0,None,1,Individual,None,None,None,0,1208,321915,4,4,2,3,3,87153,88,4,5,998,57,50800,2,15,2,10,24763,13761.0,8.3,0.0,0,163.0,378,4,3,3,4.0,NaN,4,NaN,0,2,4,4,9,27,8,14,4,13,0,0,0,6,95.0,0.0,1,0,372872,99468,15000,94072,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Cash,N,None,None,None,None,None,None
2,None,None,5000,5000,5000.0,36 months,17.97,180.69,D,D1,Administrative,6 years,MORTGAGE,59280,Source Verified,Dec-2018,Current,n,None,None,debt_consolidation,Debt consolidation,490xx,MI,10.51,0,Apr-2011,0,None,None,8,0,4599,19.1,13,w,4787.21,4787.21,353.89,353.89,212.79,141.10,0.0,0.0,0.0,Feb-2019,180.69,Mar-2019,Feb-2019,

In [5]:
# check how many records for each loan status
df_bystatus=df.groupby(df.loan_status).count()
df_bystatus.show()

+--------------------+-------+
|         loan_status|  count|
+--------------------+-------+
|          Fully Paid|1041952|
|             Default|     31|
|     In Grace Period|   8952|
|Does not meet the...|   1988|
|         Charged Off| 261654|
|            Oct-2015|      1|
|  Late (31-120 days)|  21897|
|             Current| 919695|
|Does not meet the...|    761|
|   Late (16-30 days)|   3737|
+--------------------+-------+



In [6]:
# check the number of Nan or Null for each column
#As per our objective, Filtering only the columns required
current_df = df.filter(col('loan_status').isin(['Late (31-120 days)','Charged Off','Late (16-30 days)','Current']))
#Renaming the Late values into a single late
current_df = current_df.withColumn("loan_status", \
              when(current_df["loan_status"].isin(['Late (31-120 days)','Late (16-30 days)']),'Late').otherwise(current_df["loan_status"]))
#Our dataset with only the desired columns values (Late,charged off,current)
print(current_df.groupby(current_df.loan_status).count().show())
size_shape(current_df)

+-----------+------+
|loan_status| count|
+-----------+------+
|Charged Off|261654|
|       Late| 25634|
|    Current|919695|
+-----------+------+

None
rows: 1206983
columns: 145


In [7]:
#Contains the nas present in all the columns in the dataset. We are planning to remove those coluomns with more than 50%Nas
checkna_pd = current_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in current_df.columns]).toPandas()
checkna_pd

# current_df.select([c for c in checkna_pd.columns])

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,1206983,1206983,0,0,0,0,0,0,0,0,105247,0,0,0,0,0,0,0,1206983,1187738,0,12520,1,1,1456,1,1,1,622762,1021293,5,8,6,1120,2,3,5,8,5,2,3,1,2,2,2,2414,1,261622,31,15,895784,8,12,1104847,1104856,1109686,18,10302,10299,206300,206297,206304,206307,240368,206306,355256,206310,206308,206309,206487,10316,206310,206310,206313,6884,10370,21150,21696,39,28,50004,10310,10315,10314,6885,20333,933557,147619,815085,10318,10317,10319,8263,10317,10317,10317,10318,10317,8263,58317,10318,10316,10316,10341,21262,147,32,10309,6877,6882,10314,1112427,1112425,1112422,1112422,1112425,1114082,1112428,1112429,1112428,1112428,1175616,33,1198269,1198263,1198269,1198269,1198271,1198270,1198267,1198269,1198268,1198272,1198271,1200360,1198272,1198271,34,30,1173940,1173941,1173945,1173947,1173950,1173948


In [8]:
#https://stackoverflow.com/questions/51322445/how-to-drop-all-columns-with-null-values-in-a-pyspark-dataframe - include in our references

#Creating a dataframe which has the count of Nas of each column
checkna = current_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in current_df.columns])
#Selecting the columns with NA values not greater than 50%
No_of_rows = current_df.count()
checkna_greater_than_50 = checkna.select([when(fn.col(c)<(No_of_rows*0.50),c).alias(c) for c in checkna.columns])


def drop_null_columns(df):
    """
    This function drops all columns which contain null values.
    :param df: A PySpark DataFrame
    """
    null_counts = df.select([fn.count(fn.when(fn.col(c).isNull()|isnan(fn.col(c)), c)).alias(c) for c in df.columns]).collect()[0].asDict()
    to_drop = [k for k, v in null_counts.items() if v > 0]
    df = df.drop(*to_drop)
    return df

# Using the function to find the list of columns with less than 50%Na values
final_cols = drop_null_columns(checkna_greater_than_50)
current_df1 = current_df.select([fn.col(c) for c in final_cols.columns])
size_shape(current_df)
size_shape(current_df1)
#selecting our dataframe with the final cols
# current_df1 

rows: 1206983
columns: 145
rows: 1206983
columns: 102


In [204]:
part4 = current_df1.select(current_df1.columns[76::1])
# Displaying the dataframe in pandas
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth',-1)
part4.toPandas().head(5)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,2.0,0.0,2.0,5.0,3.0,3.0,16.0,7.0,18.0,5.0,9.0,0.0,0.0,0.0,3.0,100.0,0.0,1.0,0.0,60124.0,16901.0,36500.0,18124.0,N,Cash,N
1,4.0,0.0,2.0,4.0,4.0,9.0,27.0,8.0,14.0,4.0,13.0,0.0,0.0,0.0,6.0,95.0,0.0,1.0,0.0,372872.0,99468.0,15000.0,94072.0,N,Cash,N
2,14.0,0.0,0.0,3.0,3.0,3.0,4.0,6.0,7.0,3.0,8.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,136927.0,11749.0,13800.0,10000.0,N,Cash,N
3,5.0,0.0,1.0,2.0,1.0,2.0,7.0,2.0,3.0,2.0,10.0,0.0,0.0,0.0,3.0,100.0,100.0,0.0,0.0,385183.0,36151.0,5000.0,44984.0,N,Cash,N
4,13.0,0.0,2.0,2.0,3.0,8.0,9.0,6.0,15.0,2.0,12.0,0.0,0.0,0.0,5.0,92.3,0.0,0.0,0.0,157548.0,29674.0,9300.0,32332.0,N,Cash,N


In [10]:
part4.printSchema()

root
 |-- mths_since_recent_inq: integer (nullable = true)
 |-- num_accts_ever_120_pd: integer (nullable = true)
 |-- num_actv_bc_tl: integer (nullable = true)
 |-- num_actv_rev_tl: integer (nullable = true)
 |-- num_bc_sats: integer (nullable = true)
 |-- num_bc_tl: integer (nullable = true)
 |-- num_il_tl: integer (nullable = true)
 |-- num_op_rev_tl: integer (nullable = true)
 |-- num_rev_accts: integer (nullable = true)
 |-- num_rev_tl_bal_gt_0: integer (nullable = true)
 |-- num_sats: integer (nullable = true)
 |-- num_tl_120dpd_2m: integer (nullable = true)
 |-- num_tl_30dpd: integer (nullable = true)
 |-- num_tl_90g_dpd_24m: integer (nullable = true)
 |-- num_tl_op_past_12m: integer (nullable = true)
 |-- pct_tl_nvr_dlq: double (nullable = true)
 |-- percent_bc_gt_75: double (nullable = true)
 |-- pub_rec_bankruptcies: integer (nullable = true)
 |-- tax_liens: integer (nullable = true)
 |-- tot_hi_cred_lim: integer (nullable = true)
 |-- total_bal_ex_mort: integer (nullable = tr

In [12]:
#Using count distinct to check about the columns and ascertain its type accordingly
part4.select([fn.countDistinct(c).alias(c) for c in part4.columns]).toPandas().T
 

,0
mths_since_recent_inq,26
num_accts_ever_120_pd,43
num_actv_bc_tl,41
num_actv_rev_tl,54
num_bc_sats,58
num_bc_tl,72
num_il_tl,118
num_op_rev_tl,77
num_rev_accts,111
num_rev_tl_bal_gt_0,50


In [205]:
#Using count distinct to check about the columns and ascertain its type accordingly - whether categorical,float ,integer..etc
Distinctcount4 = part4.select([fn.countDistinct(c).alias(c) for c in part4.columns])
# Distinctcount Converting to Pandas(So it computes faster)
DC4= Distinctcount4.toPandas()

In [206]:
#This is to check for categorical numerical and see if we have weird categories.()
# for col in part4.columns:
#     if DC4[col][0]<1000:
#         print(part4.groupby(col).count().show())
#     else:
#         continue

In [212]:
#All these values along with NAs should be replaced first - by the mode for categorical columns with less than 1000 distinct values
#These are taken from the above output
cols = ['hardship_flag','disbursement_method','debt_settlement_flag']
wrong_val = ['Feb-2019','Individual','Oct-2011','Oct-2016']
cleaned_int_categorical4 = []

numerical_cols4 = []
from pyspark.sql.types import *
d = {}

for col,dtype in part4.dtypes:
    
    #Checking that the distinct values less than 1000 to be qualified as a categorical where we replace the values with mode
    if DC4[col][0]<1000:
        if dtype in ['double','int']:
            print(col,dtype)
            part4 = part4.withColumn(col,fn.when(fn.col(col).isin(wrong_val),'null').otherwise(fn.col(col)))
            #Converting them to integer, since in my case.. all are integer type category
            part4 = part4.withColumn(col,part4[col].cast(IntegerType()))
            #Filling the NAs by the mode
            temp = part4.groupby(fn.col(col)).count()
            #Making sure that the grouped mode value is not a null
            mode_value = temp.orderBy(temp['count'].desc()).collect()[0][0]
            if mode_value is None:
                mode_value = temp.orderBy(temp['count'].desc()).collect()[1][0]
            part4 = part4.fillna(mode_value,subset = col)
            print(col,' done')
            cleaned_int_categorical4.append(col)
            #Winsorizing the columns
            d[col] = part4.approxQuantile(col,[0.01,0.99],0.25)
            part4 = part4.withColumn(col,fn.when(fn.col(col)<d[col][0],d[col][0]).when(fn.col(col)>d[col][1],d[col][1]).otherwise(fn.col(col)))
            print(col,'winsorising done')
        else:
            continue
                    
    else:
        #This means that they have more than 1000 distinct values and can be treated as Float continous
        print(col,dtype)
        part4 = part4.withColumn(col,fn.when(part4[col].isin(wrong_val),'null').otherwise(part4[col]))
        #Casting to float type
        part4 = part4.withColumn(col,part4[col].cast(FloatType()))
        #Replacing the float type value nas with its mean
        mean_val = part4.select(fn.avg(fn.col(col))).collect()[0][0]
        part4 = part4.fillna(mean_val,subset = col)
        #WInsorizing the values
        d[col] = part4.approxQuantile(col,[0.01,0.99],0.25)
        part4 = part4.withColumn(col,fn.when(fn.col(col)<d[col][0],d[col][0]).when(fn.col(col)>d[col][1],d[col][1]).otherwise(fn.col(col)))
        print(col,'winsorising done')
        numerical_cols4.append(col)
   
    
#These are the count of Null values in the columns
part4.select([fn.count(fn.when(fn.isnan(c)|fn.isnull(c),c)).alias(c) for c in part4.columns]).toPandas().T           


mths_since_recent_inq double
mths_since_recent_inq  done
mths_since_recent_inq winsorising done
num_accts_ever_120_pd double
num_accts_ever_120_pd  done
num_accts_ever_120_pd winsorising done
num_actv_bc_tl double
num_actv_bc_tl  done
num_actv_bc_tl winsorising done
num_actv_rev_tl double
num_actv_rev_tl  done
num_actv_rev_tl winsorising done
num_bc_sats double
num_bc_sats  done
num_bc_sats winsorising done
num_bc_tl double
num_bc_tl  done
num_bc_tl winsorising done
num_il_tl double
num_il_tl  done
num_il_tl winsorising done
num_op_rev_tl double
num_op_rev_tl  done
num_op_rev_tl winsorising done
num_rev_accts double
num_rev_accts  done
num_rev_accts winsorising done
num_rev_tl_bal_gt_0 double
num_rev_tl_bal_gt_0  done
num_rev_tl_bal_gt_0 winsorising done
num_sats double
num_sats  done
num_sats winsorising done
num_tl_120dpd_2m double
num_tl_120dpd_2m  done
num_tl_120dpd_2m winsorising done
num_tl_30dpd double
num_tl_30dpd  done
num_tl_30dpd winsorising done
num_tl_90g_dpd_24m double
nu

,0
mths_since_recent_inq,0
num_accts_ever_120_pd,0
num_actv_bc_tl,0
num_actv_rev_tl,0
num_bc_sats,0
num_bc_tl,0
num_il_tl,0
num_op_rev_tl,0
num_rev_accts,0
num_rev_tl_bal_gt_0,0


In [214]:
#Now for the string Cols - Since those are categorical, I will replace it by the value 'unknown'. Since the fraction of Null is negligible
string_cols4 = ['disbursement_method','debt_settlement_flag','hardship_flag']
part4 = part4.fillna('unknown',subset = string_cols4)
part4.select([fn.count(fn.when(fn.isnan(c)|fn.isnull(c),c)).alias(c) for c in part4.columns]).toPandas().T    

,0
mths_since_recent_inq,0
num_accts_ever_120_pd,0
num_actv_bc_tl,0
num_actv_rev_tl,0
num_bc_sats,0
num_bc_tl,0
num_il_tl,0
num_op_rev_tl,0
num_rev_accts,0
num_rev_tl_bal_gt_0,0


In [215]:
cleaned_dataframe_part4 = part4

In [216]:
part4_pd = cleaned_dataframe_part4.describe().toPandas()
part4_pd 

,summary,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,count,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983
1,mean,6.4035657503046854,0.4934452266519081,3.735859577144003,5.697646114319754,4.8142625041114915,7.348869039580507,8.251694514338645,8.261739394838203,13.42225284034655,5.6414183132653895,11.660910717052353,4.8385105672573683E-4,0.0023645734861220086,0.07935654437552145,2.0465772923065195,93.88903406261728,40.190242944598225,0.12669855333505112,0.04405613003662852,175710.3586867395,52091.863219933635,23918.073176947837,45046.47045789999,0.0,None,None
2,stddev,5.995041327316529,1.369332000248246,2.4020143875606164,3.4948934269583383,3.11741479443076,4.565229410123265,7.372163701309126,4.804796963399591,7.922998307165829,3.3990616052327605,5.795399769369285,0.024956111293652505,0.05129070501868895,0.4886935140387846,1.8500357427662086,9.391781278466562,36.43498811869198,0.3589824946288706,0.38249274914253,179480.10171713604,50956.73368087077,23635.00203682197,45993.316117107184,NaN,None,None
3,min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Cash,Cash
4,max,25.0,58.0,50.0,72.0,71.0,86.0,159.0,91.0,151.0,65.0,101.0,7.0,6.0,58.0,32.0,100.0,100.0,11.0,100.0,9999999.0,2698600.0,1569000.0,2118996.0,unknown,unknown,unknown


In [217]:
all_cols = []
# Now that cleaning is done ; Let us start with the pipeline
# Since all the categorical columns are numbers

cat_coded4_cols = []
string_cols4 = []
for c in cleaned_int_categorical4:
    print(c)
    part4 = Pipeline(stages = [feature.OneHotEncoder(inputCol = c,outputCol = c+'catcoded4')]).fit(part4).transform(part4)
    cat_coded4_cols.append(c+'catcoded4')
for c in string_cols:
    part4 = Pipeline(stages = [feature.StringIndexer(inputCol = c,outputCol = c+'index4'),
                              feature.OneHotEncoder(inputCol = c+'index4',outputCol = c+'stringcoded4')]).fit(part4).transform(part4)
    string_cols4.append(c+'stringcoded4')


mths_since_recent_inq
num_accts_ever_120_pd
num_actv_bc_tl
num_actv_rev_tl
num_bc_sats
num_bc_tl
num_il_tl
num_op_rev_tl
num_rev_accts
num_rev_tl_bal_gt_0
num_sats
num_tl_120dpd_2m
num_tl_30dpd
num_tl_90g_dpd_24m
num_tl_op_past_12m
pct_tl_nvr_dlq
percent_bc_gt_75
pub_rec_bankruptcies
tax_liens


In [219]:
total_cols4 = cat_coded4_cols + string_cols4 + numerical_cols4

#Use features4 as stages for your modelling needs
part4_pipe = Pipeline(stages = [feature.VectorAssembler(inputCols = total_cols4,outputCol = 'feature4')])
part4_pipe.fit(part4).transform(part4).first()
#For cleaned dataframe - use cleaned_dataframe_part4 ( which is just all columns cleaned )

Row(mths_since_recent_inq=2.0, num_accts_ever_120_pd=0.0, num_actv_bc_tl=2.0, num_actv_rev_tl=5.0, num_bc_sats=3.0, num_bc_tl=3.0, num_il_tl=16.0, num_op_rev_tl=7.0, num_rev_accts=18.0, num_rev_tl_bal_gt_0=5.0, num_sats=9.0, num_tl_120dpd_2m=0.0, num_tl_30dpd=0.0, num_tl_90g_dpd_24m=0.0, num_tl_op_past_12m=3.0, pct_tl_nvr_dlq=100.0, percent_bc_gt_75=0.0, pub_rec_bankruptcies=1.0, tax_liens=0.0, tot_hi_cred_lim=60124.0, total_bal_ex_mort=16901.0, total_bc_limit=36500.0, total_il_high_credit_limit=18124.0, hardship_flag='N', disbursement_method='Cash', debt_settlement_flag='N', mths_since_recent_inqcatcoded4=SparseVector(25, {2: 1.0}), num_accts_ever_120_pdcatcoded4=SparseVector(58, {0: 1.0}), num_actv_bc_tlcatcoded4=SparseVector(50, {2: 1.0}), num_actv_rev_tlcatcoded4=SparseVector(72, {5: 1.0}), num_bc_satscatcoded4=SparseVector(71, {3: 1.0}), num_bc_tlcatcoded4=SparseVector(86, {3: 1.0}), num_il_tlcatcoded4=SparseVector(159, {16: 1.0}), num_op_rev_tlcatcoded4=SparseVector(91, {7: 1.0})

In [221]:
part3 = current_df1.select(current_df1.columns[50:76:1])
part3.toPandas().head()

,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc
0,2,1,2,2,12560,69,2,7,2137,28,42000,1,11,2,9.0,1878,34360.0,5.9,0.0,0.0,140.0,212.0,1.0,1.0,0.0,1.0
1,4,2,3,3,87153,88,4,5,998,57,50800,2,15,2,10.0,24763,13761.0,8.3,0.0,0.0,163.0,378.0,4.0,3.0,3.0,4.0
2,1,0,2,14,7150,72,0,2,0,35,24100,1,5,0,4.0,18383,13800.0,0,0.0,0.0,87.0,92.0,15.0,14.0,2.0,77.0
3,5,3,5,5,30683,68,0,0,3761,70,7000,2,4,3,5.0,30505,1239.0,75.2,0.0,0.0,62.0,154.0,64.0,5.0,3.0,64.0
4,5,3,5,4,28845,89,2,4,516,54,23100,1,0,0,9.0,9667,8471.0,8.9,0.0,0.0,53.0,216.0,2.0,2.0,2.0,2.0


In [222]:
#These are the count of Null values in the columns
part3.select([fn.count(fn.when(fn.isnan(c)|fn.isnull(c),c)).alias(c) for c in part3.columns]).toPandas().T

,0
open_act_il,206297
open_il_12m,206304
open_il_24m,206307
mths_since_rcnt_il,240368
total_bal_il,206306
il_util,355256
open_rv_12m,206310
open_rv_24m,206308
max_bal_bc,206309
all_util,206487


In [223]:
#Using count distinct to check about the columns and ascertain its type accordingly - whether categorical,float ,integer..etc
Distinctcount3 = part3.select([fn.countDistinct(c).alias(c) for c in part3.columns])
# Distinctcount Converting to Pandas(So it computes faster)
DC3= Distinctcount.toPandas()

In [224]:
#This is to check for wrong values & correct them.
# for col in part3.columns:
#     if Distinctcount3.select(fn.col(col)).collect()[0][0]<1000:
#         print(part3.groupby(col).count().show())
#     else:
#         continue

In [226]:
#All these values along with NAs should be replaced first - by the mode for categorical columns with less than 1000 distinct values
#These are taken from the above output - BUT THE PROBLEM HERE IS (ONLY FROM THE TOP 20 ROWS)

wrong_val = ['Feb-2019','Individual','Oct-2011','Oct-2016']
cleaned_int_categorical3 = []

numerical_cols3 = []
from pyspark.sql.types import *
d = {}

for col,dtype in part3.dtypes:
    
    #Checking that the distinct values less than 1000 to be qualified as a categorical where we replace the values with mode
    if DC3[col][0]<1000:
        if dtype in ['string','double','int']:
            print(col,dtype)
            #COnverting those wrong values to null
            part3 = part3.withColumn(col,fn.when(fn.col(col).isin(wrong_val),'null').otherwise(fn.col(col)))
            #Converting them to integer, since in my case.. all are integer type category
            part3 = part3.withColumn(col,part3[col].cast(IntegerType()))
            #Filling the NAs by the mode
            temp = part3.groupby(fn.col(col)).count()
            #Making sure that the grouped mode value is not a null
            mode_value = temp.orderBy(temp['count'].desc()).collect()[0][0]
            if mode_value is None:
                mode_value = temp.orderBy(temp['count'].desc()).collect()[1][0]
            part3 = part3.fillna(mode_value,subset = col)
            print(col,' done')
            cleaned_int_categorical3.append(col)
            #Winsorizing the columns
            d[col] = part3.approxQuantile(col,[0.01,0.99],0.25)
            part3 = part3.withColumn(col,fn.when(fn.col(col)<d[col][0],d[col][0]).when(fn.col(col)>d[col][1],d[col][1]).otherwise(fn.col(col)))
            print(col,'winsorising done')
    else:
        #This means that they have more than 1000 distinct values and can be treated as continous
        print(col,dtype)
        part3 = part3.withColumn(col,fn.when(part3[col].isin(wrong_val),'null').otherwise(part3[col]))
        #Casting to float type
        part3 = part3.withColumn(col,part3[col].cast(FloatType()))
        #Replacing the float type value nas with its mean
        mean_val = part3.select(fn.avg(fn.col(col))).collect()[0][0]
        part3 = part3.fillna(mean_val,subset = col)
        #WInsorizing the values
        d[col] = part3.approxQuantile(col,[0.01,0.99],0.25)
        part3 = part3.withColumn(col,fn.when(fn.col(col)<d[col][0],d[col][0]).when(fn.col(col)>d[col][1],d[col][1]).otherwise(fn.col(col)))
        print(col,'winsorising done')
        numerical_cols3.append(col)
   
    
#These are the count of Null values in the columns
part3.select([fn.count(fn.when(fn.isnan(c)|fn.isnull(c),c)).alias(c) for c in part3.columns]).toPandas().T           


open_act_il double
open_act_il  done
open_act_il winsorising done
open_il_12m double
open_il_12m  done
open_il_12m winsorising done
open_il_24m double
open_il_24m  done
open_il_24m winsorising done
mths_since_rcnt_il double
mths_since_rcnt_il  done
mths_since_rcnt_il winsorising done
total_bal_il float
total_bal_il winsorising done
il_util string
il_util  done
il_util winsorising done
open_rv_12m string
open_rv_12m  done
open_rv_12m winsorising done
open_rv_24m string
open_rv_24m  done
open_rv_24m winsorising done
max_bal_bc string
max_bal_bc winsorising done
all_util string
all_util  done
all_util winsorising done
total_rev_hi_lim string
total_rev_hi_lim winsorising done
inq_fi string
inq_fi  done
inq_fi winsorising done
total_cu_tl string
total_cu_tl  done
total_cu_tl winsorising done
inq_last_12m string
inq_last_12m  done
inq_last_12m winsorising done
acc_open_past_24mths int
acc_open_past_24mths  done
acc_open_past_24mths winsorising done
avg_cur_bal string
avg_cur_bal winsorising 

,0
open_act_il,0
open_il_12m,0
open_il_24m,0
mths_since_rcnt_il,0
total_bal_il,0
il_util,0
open_rv_12m,0
open_rv_24m,0
max_bal_bc,0
all_util,0


In [191]:
part3_pd = part3_cleaned.describe().toPandas()
part3_pd

,summary,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc
0,count,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983,1206983
1,mean,2.473073771544421,0.6837834501397285,1.4215618612689658,18.87988728921617,35267.32689077014,71.23483512195284,1.042537467387693,2.532824405977549,10898.191846219044,57.470946981026245,35326.35072485486,0.8252469173136655,1.1694000661152641,1.6333635187902398,4.4989332906925785,13247.09039424126,12348.634003000663,689.7896024208974,0.015629051941908046,11.896876084833009,125.27161691589691,179.16532130112853,14.319263817303144,8.417290052966777,1.3836002661180813,24.688998105192866
2,stddev,2.846748247781309,157.640334478604,6.185888510293743,24.74873763913723,40388.74619082968,20.6617318580488,1.4795472348796592,144.12176129412308,12217.771446680843,19.012856544134063,35065.15446138508,4.649738617880482,2.4235188895512985,16.536155625819518,3.2055796606882576,16146.928759210394,17240.46842022785,4678.217083563141,8.290951261342633,710.0637004903783,53.610124939788264,99.11402965901016,18.134524431502925,9.434107281507575,1.7671889111133305,32.99331214436471
3,min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,max,421.0,173057.0,5918.0,3367.0,1837038.0,1000.0,381.0,158316.0,1170668.0,239.0,2175000.0,4878.0,111.0,18000.0,64.0,752994.0,711140.0,35267.328125,9108.0,249925.0,999.0,999.0,547.0,368.0,94.0,661.0


In [194]:
#Ah well, guess
part3.approxQuantile('open_il_12m',[0.01,0.99],0.25)

[0.0, 173057.0]

In [227]:
#Now that NAs are done.. Lets feature Engineer..
# Since all the categorical columns are numbers
part3_cleaned = part3
cat_coded3_cols = []
for c in cleaned_int_categorical:
    print(c)
    part3 = Pipeline(stages = [feature.OneHotEncoder(inputCol = c,outputCol = c+'catcoded3')]).fit(part3).transform(part3)
    cat_coded3_cols.append(c+'catcoded3')


open_act_il
open_il_12m
open_il_24m
mths_since_rcnt_il
il_util
open_rv_12m
open_rv_24m
all_util
inq_fi
total_cu_tl
inq_last_12m
acc_open_past_24mths
chargeoff_within_12_mths
mo_sin_old_il_acct
mo_sin_old_rev_tl_op
mo_sin_rcnt_rev_tl_op
mo_sin_rcnt_tl
mort_acc
mths_since_recent_bc


In [228]:
total_cols3 = cat_coded3_cols + numerical_cols3

#Use features4 as stages for your modelling needs
part3_pipe = Pipeline(stages = [feature.VectorAssembler(inputCols = total_cols3,outputCol = 'feature3')])
part3_pipe.fit(part3).transform(part3).first()

Row(open_act_il=2.0, open_il_12m=1.0, open_il_24m=2.0, mths_since_rcnt_il=2.0, total_bal_il=12560.0, il_util=69.0, open_rv_12m=2.0, open_rv_24m=7.0, max_bal_bc=2137.0, all_util=28.0, total_rev_hi_lim=42000.0, inq_fi=1.0, total_cu_tl=11.0, inq_last_12m=2.0, acc_open_past_24mths=9.0, avg_cur_bal=1878.0, bc_open_to_buy=34360.0, bc_util=5.900000095367432, chargeoff_within_12_mths=0.0, delinq_amnt=0.0, mo_sin_old_il_acct=140.0, mo_sin_old_rev_tl_op=212.0, mo_sin_rcnt_rev_tl_op=1.0, mo_sin_rcnt_tl=1.0, mort_acc=0.0, mths_since_recent_bc=1.0, open_act_ilcatcoded3=SparseVector(421, {2: 1.0}), open_il_12mcatcoded3=SparseVector(173057, {1: 1.0}), open_il_24mcatcoded3=SparseVector(5918, {2: 1.0}), mths_since_rcnt_ilcatcoded3=SparseVector(3367, {2: 1.0}), il_utilcatcoded3=SparseVector(1000, {69: 1.0}), open_rv_12mcatcoded3=SparseVector(381, {2: 1.0}), open_rv_24mcatcoded3=SparseVector(158316, {7: 1.0}), all_utilcatcoded3=SparseVector(239, {28: 1.0}), inq_ficatcoded3=SparseVector(4878, {1: 1.0}), t